# Part2: Model Training

<h3>2.1 Baseline Model </h3>

We first make a random forset model as a baseline model

In [24]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from typing import *
import logging
from data import DATA_DIR
from lama.preprocessing.DataProcessor import nans, reformat_dataframe

logger = logging.getLogger()
PRE_DIR = os.path.join(DATA_DIR, "pre")

We do not need card id anynore since we don't have to merge tables

In [3]:
train = pd.read_csv(os.path.join(PRE_DIR, "train_groupby.csv"))
test = pd.read_csv(os.path.join(PRE_DIR, "test_groupby.csv"))

features = train.columns.tolist()
features.remove('target')
features.remove('card_id')


In [4]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [5]:
# Used to find best parameter do not invoke
# PARAMETER_SPACE = {
#     "n_estimators": range(100),
#     "min_samples_leaf": range(50),
#     "min_samples_split": range(10),
#     "max_depth": range(20),
#     "max_features": ["auto", 80]
# }

In [6]:
# clf = RandomForestRegressor(criterion='squared_error', n_jobs=16, random_state=42)
# grid = GridSearchCV(clf, PARAMETER_SPACE, cv=2, scoring='neg_mean_squared_error')
# print(train['target'].values)
# grid.fit(train[features].values, train['target'].values)

In [7]:
# grid.best_params_

In [8]:
# best_score = np.sqrt(-grid.best_score_)
# _result = \
# {
#     "model": "RandomForestRegressor",
#     "best_params": grid.best_params_,
#     "best_score": best_score,
# }


In [9]:
# test['target'] = grid.best_estimator_.predict(test[features])

In [10]:
# import json
#
# RESULT_DIR = os.path.join(DATA_DIR, "result")
# test[['card_id', 'target']].to_csv(os.path.join(RESULT_DIR, "submission_random_forest_1.csv"), index=False)
# with open(os.path.join(RESULT_DIR, "hyperparameters"), "a") as file:
#     file.write(json.dumps(_result) + "\n")

After submit our first baseline model into kagggle, we get the result.

<img src="../../assets/img/Screenshot_1.png" />

We can see that this result is not very performing. So we will try out other model or make hyperparameter optimizations based on this result.

<h3>2.2 Baseline Model </h3>

Now we try to build the model the second time, by implementing PCA and other filter model, we will run our random forest model a second time, hoping it achieve a better performance

### Performing a feature selection on all features

In [11]:
X_train = train[features]
X_train.head()

,first_active_month,feature_1,feature_2,feature_3,month_lag_nunique_hist_x,month_lag_mean_hist_x,month_lag_min_hist_x,month_lag_max_hist_x,month_lag_var_hist_x,month_lag_skew_hist_x,...,category_3_nunique,category_1_nunique,merchant_category_id_nunique,subsector_id_nunique,merchant_id_nunique,city_id_nunique,state_id_nunique,category_2_nunique,most_recent_sales_range_nunique,most_recent_purchases_range_nunique
0,67,5,2,1,8,-4.291139,-8,-1,4.673353,0.053360,...,2,1,46,21,117,9,3,2,5,5
1,62,4,1,0,12,-5.889632,-12,-1,11.883751,-0.175152,...,3,2,58,24,148,9,3,2,5,5
2,57,2,2,0,11,-8.809524,-13,-1,13.279907,0.712599,...,1,1,9,8,14,5,2,2,4,5
3,70,4,3,0,5,-3.573770,-5,-1,1.415301,0.948636,...,3,2,28,15,57,7,5,3,5,5
4,72,1,3,0,3,-1.744898,-3,-1,0.624974,-0.490374,...,3,2,37,19,102,7,7,5,5,5


In [69]:
from scipy.stats import pearsonr

X_train = train.drop('card_id', axis=1)
X_test = test

X_centered = X_train.apply(lambda x: x - x.mean())
for feature in features:
    print(f'Feature {feature} with pearson correlcation: {pearsonr(X_centered[feature], X_centered["target"])[0]}')

Feature first_active_month with pearson correlcation: 0.005224044791546985
Feature feature_1 with pearson correlcation: -0.010884238458633405
Feature feature_2 with pearson correlcation: -0.003606531136815961
Feature feature_3 with pearson correlcation: -0.007330298118724083
Feature month_lag_nunique_hist_x with pearson correlcation: 0.008782863062287089
Feature month_lag_mean_hist_x with pearson correlcation: -0.038338172765549475
Feature month_lag_min_hist_x with pearson correlcation: -0.028035212700470862
Feature month_lag_max_hist_x with pearson correlcation: -0.031797733173404084
Feature month_lag_var_hist_x with pearson correlcation: 0.020122198307935886
Feature month_lag_skew_hist_x with pearson correlcation: 0.01851368587257077
Feature month_lag_sum_hist_x with pearson correlcation: 0.01734761203057366
Feature installments_nunique_hist_x with pearson correlcation: 0.009186063201157288
Feature installments_mean_hist_x with pearson correlcation: 0.02528167068876249
Feature instal

The result presents above shows that many columns has a low correlation with our target.
so we will set an appropriate dimensions to reduce the features columns

In [70]:
corr = {}

# append absolute correlations with each feature, see pearson correlation above
for fea in features:
    corr[fea] = abs(train[[fea, 'target']].fillna(0).corr().values[0][1])

corr

{'first_active_month': 0.005224044791547059,
 'feature_1': 0.010884238458633256,
 'feature_2': 0.0036065311368159952,
 'feature_3': 0.007330298118724101,
 'month_lag_nunique_hist_x': 0.008782863062287207,
 'month_lag_mean_hist_x': 0.03833817276554955,
 'month_lag_min_hist_x': 0.02803521270047124,
 'month_lag_max_hist_x': 0.031797733173403675,
 'month_lag_var_hist_x': 0.020122198307936195,
 'month_lag_skew_hist_x': 0.01851368587257108,
 'month_lag_sum_hist_x': 0.017347612030573677,
 'installments_nunique_hist_x': 0.009186063201157349,
 'installments_mean_hist_x': 0.025281670688762126,
 'installments_min_hist_x': 0.003283281627196056,
 'installments_max_hist_x': 0.013174162574504608,
 'installments_var_hist_x': 0.023876356875117702,
 'installments_skew_hist_x': 0.02254950272786105,
 'installments_sum_hist_x': 0.010939218902732377,
 'purchase_amount_nunique_hist_x': 0.034459357510949216,
 'purchase_amount_mean_hist_x': 0.00017181771449204043,
 'purchase_amount_min_hist_x': 0.0094830281282

In [71]:
corr = {k: v for k, v in sorted(corr.items(), key=lambda item: item[1], reverse=True)}
featuresSelect = list(corr.keys())[:100]

# use first 100 most significant keys
featuresSelect

['month_lag_nunique_hist_y',
 'month_lag_max',
 'most_recent_purchases_range_nunique_hist_y',
 'most_recent_sales_range_nunique_hist_y',
 'subsector_id_nunique_hist_y',
 'month_lag_sum_hist_y',
 'month_lag_max_hist_y',
 'merchant_category_id_nunique_hist_y',
 'numerical_1_nunique_hist_y',
 'numerical_2_nunique_hist_y',
 'merchant_id_nunique_hist_y',
 'purchase_amount_nunique_hist_y',
 'card_id_size_hist_y',
 'card_id_count_hist_y',
 'city_id_nunique_hist_y',
 'authorized_flag_nunique_hist_y',
 'purchase_amount_skew_hist_y',
 'state_id_nunique_hist_y',
 'subsector_id_nunique',
 'merchant_category_id_nunique',
 'month_lag_mean_hist_y',
 'numerical_2_skew_hist_y',
 'category_2_nunique_hist_y',
 'numerical_1_skew_hist_y',
 'month_lag_mean',
 'numerical_2_nunique',
 'numerical_1_nunique',
 'merchant_id_nunique',
 'installments_nunique_hist_y',
 'most_recent_purchases_range_nunique',
 'authorized_flag_nunique',
 'most_recent_sales_range_nunique',
 'category_3_nunique_hist_y',
 'installments_

In [72]:
from lama.util.decorators import enable_logging

@enable_logging("param_grid_search.log")
def param_grid_search(X_train, features, parameter_space, y_target, X_test, filename):
    """
    Grid Search Optimizer

    Returns: best_parameters

    """

    logger.info("Creating Grid Search")
    assert features is not None and 'card_id' not in features and 'target' not in features

    # Tuning hyper-parameters for mse
    clf = RandomForestRegressor(
        criterion= 'squared_error',
        n_jobs= 15,
        random_state= 22
    )

    grid = GridSearchCV(clf, parameter_space, cv=2, scoring='neg_mean_squared_error')
    grid.fit(X_train[features].values, y_target.values)

    logger.info("grid fit completes")

    logger.debug("Best parameters:")
    logger.debug(grid.best_params_)

    means = grid.cv_results_["mean_test_score"]
    stds = grid.cv_results_["std_test_score"]
    params = grid.cv_results_["params"]

    # check different value in cv validation
    for mean, std, param in means, stds, params:
        msg = "%0.03f (+/-%0.03f) for %r" %(mean, std, param)
        logger.debug(msg)

    # write to hyperparameters
    logger.info("Write to files")

    best_score = np.sqrt(-grid.best_score_)
    _result = \
    {
        "model": "RandomForestRegressor",
        "best_params": grid.best_params_,
        "best_score": best_score,
    }

    X_test['target'] = grid.best_estimator_.predict(X_test[features])

    import json

    RESULT_DIR = os.path.join(DATA_DIR, "result")
    X_test[['card_id', 'target']].to_csv(os.path.join(RESULT_DIR, filename), index=False)

    with open(os.path.join(RESULT_DIR, "hyperparameters"), "a") as file:
        file.write(json.dumps(_result) + "\n")

    return grid

In [ ]:
PARAMETER_SPACE = {
    "n_estimators": [9, 19, 29],
    "min_samples_leaf": [4, 9, 14],
    "min_samples_split": [2, 10, 20, 30],
    "max_depth": [9, 19, 29],
    "max_features": ["auto", 80]
}

counter = 0

filename = f'sample {counter}'

counter+=1

param_grid_search(X_train, featuresSelect, PARAMETER_SPACE, X_train['target'], X_test, filename)

2022-02-03 20:09:14,763 - root - INFO - Entering into the function param_grid_search.
2022-02-03 20:09:14,766 - root - INFO - Creating Grid Search
/opt/anaconda3/envs/lama/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:392: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
/opt/anaconda3/envs/lama/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:392: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
/opt/anaconda3/envs/lama/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:392: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
/opt/anaconda3/envs/lama/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:392: FutureWarning: Criterion 'mse